<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Language Modeling
  </div> 
  
<div style="
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Encoder pretraining using Masked Language Modeling task
  </div> 


  <div style="
      font-size: 15px; 
      line-height: 12px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Jean-baptiste AUJOGUE - Hybrid Intelligence
  </div> 

  
  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  December 2022
  </div>

<a id="TOC"></a>

#### Table Of Content

1. [Dataset](#data) <br>
2. [ALBERT finetuning](#albert) <br>
3. [Inference](#inference) <br>



#### Reference

- Hugginface full list of [tutorial notebooks](https://github.com/huggingface/transformers/tree/main/notebooks) (see also [here](https://huggingface.co/docs/transformers/main/notebooks#pytorch-examples))
- Huggingface full list of [training scripts](https://github.com/huggingface/transformers/tree/main/examples/pytorch)
- Huggingface [tutorial notebook](https://github.com/huggingface/notebooks/blob/main/examples/language_modeling_from_scratch.ipynb) on language models
- Huggingface [course](https://huggingface.co/course/chapter7/3?fw=tf) on language models
- Huggingface [training script](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py) on language models
- Albert [original training protocol](https://github.com/google-research/albert)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import re
import random
import copy
import string
from itertools import chain

# data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import (
    Dataset,  
    DatasetDict,
    ClassLabel, 
    Features, 
    Sequence, 
    Value,
    load_from_disk,
)
from transformers import AlbertConfig, AutoConfig, DataCollatorForLanguageModeling

# DL
import torch
from gensim.models import Word2Vec
import transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    TrainingArguments, 
    Trainer,
    pipeline,
    set_seed,
)
import evaluate

# viz
from IPython.display import HTML

C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Transformers settings

In [3]:
transformers.__version__

'4.22.2'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
# make training deterministic
set_seed(42)

#### Custom paths & imports

In [6]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_data = os.path.join(path_to_repo, 'datasets', 'clinical trials CTTI')
path_to_save = os.path.join(path_to_repo, 'saves', 'MLM')
path_to_src  = os.path.join(path_to_repo, 'src')

In [7]:
sys.path.insert(0, path_to_src)

#### Constants

In [8]:
dataset_name = 'clinical-trials-ctti'
final_dataset_name = 'clinical-trials-ctti-tokenized-blocks'
base_model_name = "albert-base-v2"
final_model_name = "albert-small-ctti"

<a id="data"></a>

# 1. Dataset

[Table of content](#TOC)

We generate a collection of instances of the `datasets.Dataset` class. 

Note that these are different from the fairly generic `torch.utils.data.Dataset` class. 

## 1.1 Load Clinical Trials corpus

[Table of content](#TOC)

In [9]:
with open(os.path.join(path_to_data, '{}.txt'.format(dataset_name)), 'r', encoding = 'utf-8') as f:
    texts = [t.strip() for t in f.readlines()]

In [10]:
dataset = Dataset.from_dict({'text': texts}, features = Features({'text': Value(dtype = 'string')}))

In [11]:
len(dataset)

430108

In [12]:
dataset[:3]

{'text': ['This study will test the ability of extended release nifedipine (Procardia XL), a blood pressure medication, to permit a decrease in the dose of glucocorticoid medication children take to treat congenital adrenal hyperplasia (CAH). This protocol is designed to assess both acute and chronic effects of the calcium channel antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with congenital adrenal hyperplasia. The multicenter trial is composed of two phases and will involve a double-blind, placebo-controlled parallel design. The goal of Phase I is to examine the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (ACTH) levels, as well as to begin to assess the dose-dependency of nifedipine effects. The goal of Phase II is to evaluate the long-term effects of nifedipine; that is, can attenuation of ACTH release by nifedipine permit a decrease in the dosage of glucocorticoid needed to suppress the HPA axis? Such a decrease would, 

## 1.2 Load Clinical-Albert-small tokenizer

[Table of content](#TOC)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(path_to_save, final_model_name, 'tokenizer'))

## 1.3 Tokenize corpus

[Table of content](#TOC)

In [14]:
# We use this option because DataCollatorForLanguageModeling (see below) is more efficient 
# when it receives the `special_tokens_mask`.
def tokenize_text(examples, tokenizer):
    # Remove empty lines
    examples['text'] = [
        t for t in examples['text'] if len(t) > 0 and not t.isspace()
    ]
    return tokenizer(examples["text"], return_special_tokens_mask = True)

In [15]:
tokenized_dataset = dataset.map(
    lambda examples: tokenize_text(examples, tokenizer), 
    batched = True, 
    remove_columns = ["text"],
)

100%|████████████████████████████████████████████████████████████████████████████████| 431/431 [03:27<00:00,  2.08ba/s]


By contrast to the generic BIO annotated data, this new data depends on the tokenizer, and is therefore _model-specific_.

_Note_: the argument `remove_columns = ["text"]` is mandatory, in order to have each item of the dataset have same length.

In [16]:
len(tokenized_dataset[0]['input_ids'])

280

In [17]:
tokenized_dataset[0]

{'input_ids': [2,
  38,
  28,
  22,
  122,
  7,
  5,
  233,
  9,
  2023,
  1289,
  9167,
  92,
  20,
  5,
  19,
  922,
  7723,
  5,
  125,
  35,
  18,
  6,
  5,
  11,
  80,
  245,
  318,
  6,
  13,
  4364,
  5,
  11,
  580,
  15,
  7,
  106,
  9,
  6067,
  318,
  172,
  479,
  13,
  844,
  1636,
  4276,
  5496,
  5,
  19,
  31,
  11,
  57,
  18,
  8,
  38,
  285,
  25,
  5,
  470,
  13,
  161,
  182,
  223,
  12,
  174,
  144,
  9,
  7,
  1486,
  3565,
  2371,
  6,
  9167,
  92,
  20,
  6,
  41,
  7,
  7988,
  14,
  4757,
  14,
  11,
  16,
  4427,
  3299,
  15,
  29,
  23,
  1636,
  4276,
  5496,
  8,
  7,
  843,
  98,
  25,
  3625,
  16,
  9,
  108,
  3045,
  12,
  22,
  685,
  5,
  11,
  478,
  14,
  658,
  6,
  204,
  14,
  783,
  1151,
  330,
  8,
  7,
  640,
  9,
  152,
  5,
  21,
  25,
  13,
  546,
  7,
  5,
  233,
  9,
  9167,
  92,
  20,
  5,
  66,
  10,
  8,
  204,
  13,
  580,
  8602,
  4650,
  4583,
  889,
  5,
  19,
  1217,
  57,
  18,
  240,
  6,
  43,
  179,
  43,
  13,
 

## 1.4 Form blocks of constant length

[Table of content](#TOC)


In [18]:
def group_texts(examples, block_size):
    # Concatenate all texts.
    keys = [k for k in examples.keys() if k != 'text']
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[keys[0]])
    
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    
    # Split by chunks of max_len.
    result = {
        k: [t[i : i+block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
block_size = 512

In [20]:
mlm_dataset = tokenized_dataset.map(lambda examples: group_texts(examples, block_size), batched = True)
mlm_dataset.save_to_disk(os.path.join(path_to_data, final_dataset_name))

100%|████████████████████████████████████████████████████████████████████████████████| 430/430 [06:40<00:00,  1.07ba/s]


In [21]:
mlm_dataset = load_from_disk(os.path.join(path_to_data, final_dataset_name))

In [22]:
len(mlm_dataset)

430458

In [23]:
print(mlm_dataset[0])

{'input_ids': [2, 38, 28, 22, 122, 7, 5, 233, 9, 2023, 1289, 9167, 92, 20, 5, 19, 922, 7723, 5, 125, 35, 18, 6, 5, 11, 80, 245, 318, 6, 13, 4364, 5, 11, 580, 15, 7, 106, 9, 6067, 318, 172, 479, 13, 844, 1636, 4276, 5496, 5, 19, 31, 11, 57, 18, 8, 38, 285, 25, 5, 470, 13, 161, 182, 223, 12, 174, 144, 9, 7, 1486, 3565, 2371, 6, 9167, 92, 20, 6, 41, 7, 7988, 14, 4757, 14, 11, 16, 4427, 3299, 15, 29, 23, 1636, 4276, 5496, 8, 7, 843, 98, 25, 3625, 16, 9, 108, 3045, 12, 22, 685, 5, 11, 478, 14, 658, 6, 204, 14, 783, 1151, 330, 8, 7, 640, 9, 152, 5, 21, 25, 13, 546, 7, 5, 233, 9, 9167, 92, 20, 5, 66, 10, 8, 204, 13, 580, 8602, 4650, 4583, 889, 5, 19, 1217, 57, 18, 240, 6, 43, 179, 43, 13, 1737, 13, 161, 7, 106, 14, 5132, 30, 9, 9167, 92, 20, 144, 8, 7, 640, 9, 152, 5, 21, 21, 25, 13, 119, 7, 302, 14, 424, 144, 9, 9167, 92, 20, 62, 50, 25, 6, 101, 7615, 9, 1394, 57, 1289, 48, 9167, 92, 20, 4364, 5, 11, 580, 15, 7, 1704, 9, 6067, 5, 753, 13, 5, 10, 163, 39, 4495, 7, 5, 57, 39, 11, 3299, 1161, 1

In [24]:
print(tokenizer.decode(mlm_dataset[0]["input_ids"]), tokenizer.decode(mlm_dataset[0]["labels"]))

[CLS] this study will test the ability of extended release nifedipine (procardia xl), a blood pressure medication, to permit a decrease in the dose of glucocorticoid medication children take to treat congenital adrenal hyperplasia (cah). this protocol is designed to assess both acute and chronic effects of the calcium channel antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with congenital adrenal hyperplasia. the multicenter trial is composed of two phases and will involve a double-blind, placebo-controlled parallel design. the goal of phase i is to examine the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (acth) levels, as well as to begin to assess the dose-dependency of nifedipine effects. the goal of phase ii is to evaluate the long-term effects of nifedipine; that is, can attenuation of acth release by nifedipine permit a decrease in the dosage of glucocorticoid needed to suppress the hpa axis? such a decrease would, in tu

<a id="albert"></a>

# 2. ALBERT-small training

[Table of content](#TOC)

#### Tested combinations

- 1.4M parameter model: converges fast (1 epoch) towards confusion score~=2.2. Issue : Finetuning of NER on Chia hard, stuck to high training error and/or provides evaluation errors
- 3.5M parameter model: gets stuck at confusion score~=5.9. Training args : block_size = 512, bs = 16, lr = 1e-4, grad_acc_step = 4, warmup_step = 500, num_layer = 8

## 2.1 Build Clinical-Albert-small model

[Table of content](#TOC)

In [25]:
# original Albert config
config = AutoConfig.from_pretrained(base_model_name)
config

AlbertConfig {
  "_name_or_path": "albert-base-v2",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "vocab_size": 30000
}

In [26]:
# # a smaller Albert config
config = AlbertConfig(
    attention_probs_dropout_prob = 0,
    pad_token_id = 0,
    bos_token_id = 2,
    eos_token_id = 3,
    classifier_dropout_prob = 0.1,
    down_scale_factor = 1,
    embedding_size = 128,
    gap_size = 0,
    hidden_act = 'gelu_new',
    hidden_dropout_prob = 0,
    hidden_size = 512, # 768,
    initializer_range = 0.02,
    inner_group_num = 1,
    intermediate_size = 2048, # 3072,
    layer_norm_eps = 1e-12,
    max_position_embeddings = 512,
    model_type = 'albert',
    net_structure_type = 0,
    num_attention_heads = 8, # 12
    num_hidden_groups = 1,
    num_hidden_layers = 8, # 12
    num_memory_blocks = 0,
    position_embedding_type = 'absolute',
    transformers_version = '4.22.2',
    type_vocab_size = 2,
    vocab_size = 10000, # 30000,
)
model = AutoModelForMaskedLM.from_config(config)

In [27]:
model.num_parameters()

4640400

In [28]:
model = model.to(device)

## 2.2 Load pre-trained token embedding matrix (optional)

[Table of content](#TOC)

In [29]:
wv = Word2Vec.load(os.path.join(path_to_save, final_model_name, 'w2v', 'sgram')).wv

In [30]:
# reindex rows in embedding table
base_id2w = {v: k for k, v in tokenizer.get_vocab().items()}
reindexing = [wv.key_to_index[base_id2w[i]] for i in range(len(base_id2w))]

In [31]:
reindexing[:5]

[9997, 9996, 9995, 9998, 9999]

In [32]:
# for i, reind_i in enumerate(reindexing[:5]):
#     print(reind_i)
#     print(wv.index_to_key[reind_i])
#     print(base_id2w[i])

In [33]:
token_embeddings = wv.vectors[reindexing] # wv.get_normed_vectors()[reindexing]
token_embeddings.shape

(10000, 128)

In [34]:
model.albert.embeddings.word_embeddings._parameters['weight'].norm(dim = -1).tolist()

[0.0,
 0.23753541707992554,
 0.2002486288547516,
 0.2214699238538742,
 0.2249389886856079,
 0.2453431785106659,
 0.2322477251291275,
 0.2360863834619522,
 0.2506087124347687,
 0.20837904512882233,
 0.22537213563919067,
 0.21514122188091278,
 0.21907983720302582,
 0.20976503193378448,
 0.19614046812057495,
 0.22575584053993225,
 0.22946205735206604,
 0.23833496868610382,
 0.23604823648929596,
 0.2245410680770874,
 0.22522364556789398,
 0.20770332217216492,
 0.2396446317434311,
 0.2121640294790268,
 0.22122779488563538,
 0.2531817555427551,
 0.2286914587020874,
 0.23302727937698364,
 0.22124448418617249,
 0.22076423466205597,
 0.20505684614181519,
 0.20921650528907776,
 0.21449294686317444,
 0.2127392590045929,
 0.23835673928260803,
 0.22037458419799805,
 0.21465235948562622,
 0.22650131583213806,
 0.20696525275707245,
 0.23218993842601776,
 0.2226639688014984,
 0.22681188583374023,
 0.239369198679924,
 0.2163335084915161,
 0.2462770789861679,
 0.20208749175071716,
 0.22105298936367035,


In [35]:
np.linalg.norm(token_embeddings, axis = -1).tolist()

[0.051318228244781494,
 0.05024795979261398,
 1.8852003812789917,
 0.04846731945872307,
 0.052062246948480606,
 1.4907070398330688,
 1.6635342836380005,
 1.7196722030639648,
 1.48988938331604,
 1.8163033723831177,
 1.79470956325531,
 1.6517850160598755,
 1.6456834077835083,
 2.0286712646484375,
 2.1574385166168213,
 1.897613525390625,
 2.019554615020752,
 1.7978315353393555,
 1.7077692747116089,
 2.109459638595581,
 1.933674693107605,
 1.8845109939575195,
 2.3361778259277344,
 1.9730669260025024,
 2.210378408432007,
 2.3938069343566895,
 1.9208117723464966,
 2.128443717956543,
 2.3255093097686768,
 2.221428155899048,
 2.269688367843628,
 1.9382851123809814,
 1.949000597000122,
 2.2436838150024414,
 2.009927749633789,
 2.074592351913452,
 1.938167691230774,
 1.9612109661102295,
 2.2629809379577637,
 1.9347811937332153,
 2.143207550048828,
 2.1711978912353516,
 2.0664076805114746,
 2.2300586700439453,
 2.0631351470947266,
 2.1501216888427734,
 2.4426355361938477,
 2.367422103881836,
 2.2

In [36]:
model.albert.embeddings.word_embeddings = model.albert.embeddings.word_embeddings.from_pretrained(torch.tensor(token_embeddings), padding_idx = tokenizer._pad_token_type_id)

## 2.3 Model training

[Table of content](#TOC)

`Albert-vase-v2` training parameters as provided in https://github.com/google-research/albert/blob/master/run_pretraining.py : 
- max_predictions_per_seq = `20`
- train_batch_size = `4096`
- optimizer = `"lamb"`
- learning_rate = `0.00176`
- poly_power = `1.0`
- num_train_steps = `125000`
- num_warmup_steps = `3125`
- start_warmup_step = `0`
- iterations_per_loop = `1000`

The original optimizer is `lamb`, which was designed for very large batch size, see the [Lamb paper](https://arxiv.org/pdf/1904.00962.pdf), but we use here the default [AdamW](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.AdamW) optimizer with [linear learning rate decay](https://huggingface.co/docs/transformers/v4.23.1/en/main_classes/optimizer_schedules#transformers.get_linear_schedule_with_warmup), as specified in the [Trainer class documentation](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.optimizers). See the [AdamW paper](https://arxiv.org/pdf/1711.05101.pdf).

In [37]:
batch_size = 16 # <= 15 for batch_size = 512 and GPU RAM = 8GB
# batch_size = 32

In [38]:
model = model.train()

In [39]:
args = TrainingArguments(
    os.path.join(path_to_save, '_checkpoints'),
    evaluation_strategy = "no",
    learning_rate = 5e-4,
    num_train_epochs = 3,
    warmup_steps = 1500,
    gradient_accumulation_steps = 1,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_strategy = 'no',
    logging_steps = 100,
    seed = 42,
    data_seed = 23,
)

In [40]:
trainer = Trainer(
    model,
    args,
    data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15),
    train_dataset = mlm_dataset,
)

Some remarks:

- The `data_collator` is the object used to batch elements of the training & evaluation datasets.
- The `tokenizer` is provided in order to automatically pad the inputs to the maximum length when batching inputs, and to have it saved along the model, which makes it easier to rerun an interrupted training or reuse the fine-tuned model.

In [41]:
torch.cuda.empty_cache()

In [ ]:
# word2vec_epoch = 10, block_size = 512, lr = 5e-4, bs = 16, epoch = 3
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 430458
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 80712
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encod

Step,Training Loss
100,8.966300
200,8.359500
300,7.461500
400,6.589500
500,6.223500
600,6.177900
700,6.138100
800,6.103800
900,6.073800
1000,6.021000


In [42]:
# word2vec_epoch = 10, block_size = 128, lr = 5e-4, bs = 32, epoch = 3
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1722419
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 161478
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to enc

Step,Training Loss
100,8.975000
200,8.384600
300,7.533900
400,6.651100
500,6.224500
600,6.192900
700,6.149200
800,6.124800
900,6.074800
1000,6.040700


KeyboardInterrupt: 

In [38]:
# word2vec_epoch = 3, block_size = 512, lr = 5e-4, bs = 12, epoch = 2
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 430458
  Num Epochs = 2
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 71744
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encod

Step,Training Loss
100,8.975300
200,8.394600
300,7.523900
400,6.659500
500,6.238000
600,6.186700
700,6.158600
800,6.132500
900,6.081900
1000,6.047200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=71744, training_loss=2.6739787279123926, metrics={'train_runtime': 18202.4886, 'train_samples_per_second': 47.297, 'train_steps_per_second': 3.941, 'total_flos': 1.2098621094690816e+16, 'train_loss': 2.6739787279123926, 'epoch': 2.0})

In [41]:
# word2vec_epoch = 3, block_size = 512, lr = 5e-4, bs = 16, epoch = 3
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 471374
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 88383
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encod

Step,Training Loss
100,8.293300
200,7.702200
300,6.874900
400,6.151000
500,5.934700
600,5.909700
700,5.871400
800,5.839800
900,5.801800
1000,5.735500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=88383, training_loss=2.3243570542808394, metrics={'train_runtime': 21557.1365, 'train_samples_per_second': 65.599, 'train_steps_per_second': 4.1, 'total_flos': 1.0942092842876928e+16, 'train_loss': 2.3243570542808394, 'epoch': 3.0})

In [43]:
model = model.to('cpu')

In [44]:
model.save_pretrained(os.path.join(path_to_save, final_model_name, 'model'))

Configuration saved in C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\MLM\albert-small-ctti\model\config.json
Model weights saved in C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\MLM\albert-small-ctti\model\pytorch_model.bin


<a id="inference"></a>

# 3. Inference

[Table of content](#TOC)

In [32]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(path_to_save, final_model_name, 'tokenizer'))
model = AutoModelForMaskedLM.from_pretrained(os.path.join(path_to_save, final_model_name, 'model'))

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file C:\Users\jb\Desktop\NLP\Internal - Transformers for NLP\saves\MLM\clinical-trials-albert-small\model\config.json
Model config AlbertConfig {
  "_name_or_path": "C:\\Users\\jb\\Desktop\\NLP\\Internal - Transformers for NLP\\saves\\MLM\\clinical-trials-albert-small\\model",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.05,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.05,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 8,
  "n

In [45]:
mlm = pipeline(
    task = 'fill-mask', 
    model = model, 
    tokenizer = tokenizer,
    framework = 'pt',
)

In [46]:
sent = 'Polyneuropathy of other causes, including but not limited to hereditary demyelinating neuropathies, neuropathies secondary to infection or systemic disease, diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor neuropathy, monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory CIDP and acquired demyelinating symmetric (DADS) neuropathy (also known as distal CIDP).'
sent = f'Polyneuropathy of other causes, including but not limited to {mlm.tokenizer.mask_token} demyelinating neuropathies,  {mlm.tokenizer.mask_token} secondary to infection or systemic {mlm.tokenizer.mask_token}, diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor {mlm.tokenizer.mask_token}, monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory CIDP and acquired demyelinating symmetric (DADS) neuropathy (also known as distal CIDP).'
mlm(sent, top_k = 5)

[[{'score': 0.1981992870569229,
   'token': 7,
   'token_str': 'the',
   'sequence': '[CLS] polyneuropathy of other causes, including but not limited to the demyelinating neuropathies,[MASK] secondary to infection or systemic[MASK], diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor[MASK], monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory cidp and acquired demyelinating symmetric (dads) neuropathy (also known as distal cidp).[SEP]'},
  {'score': 0.08502912521362305,
   'token': 99,
   'token_str': 'other',
   'sequence': '[CLS] polyneuropathy of other causes, including but not limited to other demyelinating neuropathies,[MASK] secondary to infection or systemic[MASK], diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor[MASK], monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory cidp and acquired demyelinating symmetric (dads) neuropathy (also

[Table of content](#TOC)